In [131]:
import numpy as np
import pandas as pd
from IPython.display import display
import gc
import pyreadstat
from sklearn.preprocessing import MinMaxScaler
from semopy import Model, calc_stats, semplot, report

In [132]:
# 1. 데이터 불러오기 (CSV 파일 읽기)
kyrbs = pd.read_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\kyrbs2024.csv', encoding='cp949')
pop = pd.read_csv('c:\\data\\education\\kyrbs2024_sas\\kyrbs\\pop24.csv', encoding='cp949')

display(kyrbs.head(3))
display(pop.head(3))

,OBS,mod_d,YEAR,CITY,CTYPE,CTYPE_SD,MH,SCHOOL,STYPE,STRATA,...,E_LT_F,E_LT_SF,E_LT_M,E_LT_SM,E_EDU_F,E_KRN_F,E_BORN_F,E_EDU_M,E_KRN_M,E_BORN_M
0,A100001,2024.11.26.,2024.0,서울,대도시,대도시,중학교,중학교,남녀공학,2024_64,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
1,A100002,2024.11.26.,2024.0,서울,대도시,대도시,중학교,중학교,남녀공학,2024_64,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0
2,A100003,2024.11.26.,2024.0,서울,대도시,대도시,중학교,중학교,남녀공학,2024_64,...,1.0,9999.0,1.0,9999.0,3.0,1.0,9999.0,3.0,1.0,9999.0


,db_y,_total_,STRATA
0,2024.0,43.0,2024_1
1,2024.0,57.0,2024_10
2,2024.0,35.0,2024_100


In [133]:
# 2. 결측치 있는 열 확인
missing_cols = kyrbs.columns[kyrbs.isna().any()].tolist()
print("Missing cols:", missing_cols)
display(kyrbs[missing_cols].isna().sum())

Missing cols: ['SCHOOL', 'PA_SWD_S', 'PA_SWD_N', 'PA_SWK_S', 'PA_SWK_N', 'HT', 'WT', 'M_SLP_HR', 'M_SLP_MM', 'M_WK_HR', 'M_WK_MM', 'AC_FAGE', 'TC_FAGE', 'TC_DAGE', 'TC_EC_FAGE', 'TC_EC_DAGE', 'TC_HTP_FAGE', 'TC_HTP_DAGE', 'S_FAGE', 'INT_SPWD_TM', 'INT_SPWK_TM', 'AGE', 'AGE_M', 'E_S_RCRD', 'E_SES', 'E_RES']


SCHOOL          441
PA_SWD_S        967
PA_SWD_N        967
PA_SWK_S       1004
PA_SWK_N       1004
HT             1480
WT             1480
M_SLP_HR       4140
M_SLP_MM       4140
M_WK_HR        4140
M_WK_MM        4140
AC_FAGE          59
TC_FAGE          87
TC_DAGE          32
TC_EC_FAGE       30
TC_EC_DAGE       23
TC_HTP_FAGE      32
TC_HTP_DAGE      15
S_FAGE          155
INT_SPWD_TM    1821
INT_SPWK_TM    1855
AGE              49
AGE_M            49
E_S_RCRD          3
E_SES             5
E_RES             7
dtype: int64

In [134]:
# 3. 파생 변수 생성
kyrbs['disturbed'] = kyrbs[[f'M_GAD_{i}' for i in range(1, 8)]].apply(lambda x: 2 if (x >= 3).any() else 1, axis=1)
display(kyrbs['disturbed'].value_counts())

disturbed
1    35952
2    18701
Name: count, dtype: int64

In [135]:
kyrbs['violent'] = kyrbs['V_TRT'].apply(lambda x: 1 if x == 1 else 2)
display(kyrbs['violent'].value_counts())

violent
1    53249
2     1404
Name: count, dtype: int64

In [136]:
kyrbs = kyrbs[kyrbs['E_FM_F_1'] != 8888]

 'E_FM_F_1',
 'E_FM_SF_2',
 'E_FM_M_3',
 'E_FM_SM_4',
 'E_FM_GF_5',
 'E_FM_GM_6',
 'E_FM_OBS_7',
 'E_FM_YBS_8',
 'E_FM_NO_9',
 'E_LT_F',
 'E_LT_SF',
 'E_LT_M',
 'E_LT_SM',

In [137]:
kyrbs['E_RES'].value_counts()

E_RES
1.0    46541
4.0     1593
2.0      241
3.0      213
5.0      148
Name: count, dtype: int64

In [138]:
def next_of_kin(row): # 보호자
    if (row['E_FM_F_1'] == 1 or row['E_FM_M_3'] == 3): return 1 # 친부모
    # elif (row['E_FM_SF_2'] == 2 or row['E_FM_SM_4'] == 4): return 2 # 양부모
    # elif (row['E_FM_GF_5'] == 5 or row['E_FM_GM_6'] == 6): return 3 # 조부모
    # elif (row['E_FM_OBS_7'] == 7): return 4 # 형제자매
    else: return 5 # 없음

def livingwith(row):
    if (row['E_LT_F'] == 1 or row['E_LT_M'] == 1): return 1 # 친부모
    # elif (row['E_LT_SF'] == 1 or row['E_LT_SM'] == 1): return 2 # 양부모
    else: return 3 # 기타

kyrbs['nextofkin'] = kyrbs.apply(next_of_kin, axis=1)
kyrbs['livingwith'] = kyrbs.apply(livingwith, axis=1)

display(kyrbs['nextofkin'].value_counts())
display(kyrbs['livingwith'].value_counts())


nextofkin
1    48398
5      338
Name: count, dtype: int64

livingwith
1    47687
3     1049
Name: count, dtype: int64

In [139]:
# 4. 사용할 변수 선택 및 결측치 제거
# 모형에 사용된 관측 변수 목록
observed_vars = [
    'M_STR', 'M_SAD', 'M_LON', 'disturbed', 
    'M_SUI_CON', 
    'E_S_RCRD', 'E_SES', 'nextofkin', 'violent'
    ]

usecols = ['MH'] + observed_vars
kyrbs = kyrbs[usecols]
display(kyrbs.head(3))
kyrbs = kyrbs.dropna()

,MH,M_STR,M_SAD,M_LON,disturbed,M_SUI_CON,E_S_RCRD,E_SES,nextofkin,violent
0,중학교,4.0,1.0,2.0,1,1.0,1.0,1.0,1,1
1,중학교,3.0,2.0,2.0,1,1.0,4.0,2.0,1,1
2,중학교,2.0,1.0,3.0,1,1.0,1.0,1.0,1,1


In [140]:
# 5. 변수 척도 변환
# 스트레스 척도 변환 (1(많이) ~ 5(전혀) -> 5(많이) ~ 1(전혀))
kyrbs['M_STR'] = 6 - kyrbs['M_STR']

# 학업 성적 척도 변환 (1(상) ~ 5(하) -> 5(상) ~ 1(하))
kyrbs['E_S_RCRD'] = 6 - kyrbs['E_S_RCRD']

# 경제 상태 척도 변환 (1(상) ~ 5(하) -> 5(상) ~ 1(하))
kyrbs['E_SES'] = 6 - kyrbs['E_SES']


In [141]:
# 6. 숫자형 변수 스케일링 (MinMaxScaler)
scaler = MinMaxScaler()
cols_to_scale = ['M_STR', 'M_LON', 'E_S_RCRD', 'E_SES', 'violent']
kyrbs[cols_to_scale] = scaler.fit_transform(kyrbs[cols_to_scale])
display(kyrbs.head())

,MH,M_STR,M_SAD,M_LON,disturbed,M_SUI_CON,E_S_RCRD,E_SES,nextofkin,violent
0,중학교,0.25,1.0,0.25,1,1.0,1.00,1.00,1,0.0
1,중학교,0.50,2.0,0.25,1,1.0,0.25,0.75,1,0.0
2,중학교,0.75,1.0,0.50,1,1.0,1.00,1.00,1,0.0
3,중학교,1.00,2.0,1.00,2,2.0,0.75,0.75,1,0.0
4,중학교,0.50,1.0,0.50,1,1.0,0.75,0.50,1,0.0


In [142]:
# 7. 변수 자료형 설정 (숫자형/범주형)
numeric_cols = ['M_STR', 'M_LON', 'E_S_RCRD', 'E_SES', 'violent']
for col in numeric_cols:
    kyrbs[col] = pd.to_numeric(kyrbs[col], errors='coerce')

# 범주형 변수
categorical_cols = ['M_SAD', 'disturbed', 'M_SUI_CON', 
                    'nextofkin']
for col in categorical_cols:
    kyrbs[col] = kyrbs[col].astype("category")

In [143]:
# 8. 학교 유형별 데이터 분할 (예: 중학교와 고등학교)
mid = kyrbs[kyrbs['MH'] == '중학교']
high = kyrbs[kyrbs['MH'] == '고등학교']
print("중학교:", len(mid), "고등학교:", len(high))

중학교: 26323 고등학교: 22413


In [144]:
# 9. SEM 모형 기술 (모형 예시)
model_desc = """
    stress =~ M_STR
    mood =~ M_SAD + M_LON + disturbed
    grade =~ E_S_RCRD
    family =~ E_SES + nextofkin
    suicide =~ M_SUI_CON
    
    stress ~ mood + grade + family
    suicide ~ stress
"""


m = Model(model_desc)

In [145]:
if 'wt' not in high.columns:
    high = high.copy() # high가 view일지 copy일지 몰라 dataframe임을 확실히 하기 위한 .copy()
    high.loc[:, 'wt'] = 1.0

In [146]:
# 11. 가중 공분산 행렬 계산 함수
def weighted_cov(X, weights):
    average = np.average(X, axis=0, weights=weights)
    X_centered = X - average
    cov_matrix = np.dot((X_centered * weights[:, None]).T, X_centered) / (weights.sum() -1) # 표본집단 이므로 -1
    return cov_matrix

# 모형에 사용된 관측 변수들을 대상으로 가중 공분산 행렬 계산
data_for_cov = high[observed_vars].apply(pd.to_numeric, errors='coerce')
weights = high['wt'].to_numpy()

w_cov = weighted_cov(data_for_cov.to_numpy(), weights)
w_cov_df = pd.DataFrame(w_cov, index=observed_vars, columns=observed_vars)


In [147]:
import numpy as np

print("공분산 행렬 랭크:", np.linalg.matrix_rank(w_cov_df))
print("공분산 행렬 크기:", w_cov_df.shape)
print("데이터 샘플 개수:", high.shape[0])
print("가중치 합:", weights.sum())

공분산 행렬 랭크: 9
공분산 행렬 크기: (9, 9)
데이터 샘플 개수: 22413
가중치 합: 22413.0


In [148]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 공분산 행렬을 계산할 변수 선택
X = high[observed_vars].apply(pd.to_numeric, errors='coerce').dropna()
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)


     Feature        VIF
0      M_STR  11.018051
1      M_SAD  12.145712
2      M_LON   5.029677
3  disturbed  12.331291
4  M_SUI_CON  14.137219
5   E_S_RCRD   4.096325
6      E_SES   7.470591
7  nextofkin   7.799197
8    violent   1.027546


In [149]:
corr_matrix = high[observed_vars].corr()
display(corr_matrix)

,M_STR,M_SAD,M_LON,disturbed,M_SUI_CON,E_S_RCRD,E_SES,nextofkin,violent
M_STR,1.000000,0.360205,0.470348,0.468119,0.300905,-0.029989,-0.071152,0.004093,0.031513
M_SAD,0.360205,1.000000,0.421240,0.352021,0.376091,-0.072328,-0.049758,0.025556,0.055263
M_LON,0.470348,0.421240,1.000000,0.435740,0.335365,-0.046954,-0.102188,-0.017081,0.028221
disturbed,0.468119,0.352021,0.435740,1.000000,0.301007,-0.034377,-0.070255,0.005221,0.028866
M_SUI_CON,0.300905,0.376091,0.335365,0.301007,1.000000,-0.048853,-0.069374,0.018550,0.087490
E_S_RCRD,-0.029989,-0.072328,-0.046954,-0.034377,-0.048853,1.000000,0.277488,-0.002132,0.007868
E_SES,-0.071152,-0.049758,-0.102188,-0.070255,-0.069374,0.277488,1.000000,-0.017246,-0.001599
nextofkin,0.004093,0.025556,-0.017081,0.005221,0.018550,-0.002132,-0.017246,1.000000,0.106255
violent,0.031513,0.055263,0.028221,0.028866,0.087490,0.007868,-0.001599,0.106255,1.000000


In [150]:
display(w_cov_df)

,M_STR,M_SAD,M_LON,disturbed,M_SUI_CON,E_S_RCRD,E_SES,nextofkin,violent
M_STR,0.054117,0.037434,0.029187,0.051684,0.022488,-0.001947,-0.003534,0.000328,0.000909
M_SAD,0.037434,0.199573,0.050197,0.074637,0.053974,-0.009016,-0.004746,0.003927,0.003061
M_LON,0.029187,0.050197,0.071154,0.055165,0.028738,-0.003495,-0.005820,-0.001567,0.000933
disturbed,0.051684,0.074637,0.055165,0.225254,0.045894,-0.004553,-0.007119,0.000852,0.001699
M_SUI_CON,0.022488,0.053974,0.028738,0.045894,0.103202,-0.004379,-0.004758,0.002050,0.003485
E_S_RCRD,-0.001947,-0.009016,-0.003495,-0.004553,-0.004379,0.077856,0.016532,-0.000205,0.000272
E_SES,-0.003534,-0.004746,-0.005820,-0.007119,-0.004758,0.016532,0.045588,-0.001267,-0.000042
nextofkin,0.000328,0.003927,-0.001567,0.000852,0.002050,-0.000205,-0.001267,0.118334,0.004532
violent,0.000909,0.003061,0.000933,0.001699,0.003485,0.000272,-0.000042,0.004532,0.015373


In [151]:
# 12. 가중 공분산 행렬을 이용해 SEM 모형 적합
m.fit(high, cov=w_cov_df)

SolverResult(fun=np.float64(0.033339995744265494), success=np.True_, n_it=38, x=array([ 7.04606721e-01,  1.15232872e+00, -2.78014798e-02,  5.90073838e-01,
        1.75594451e-02, -3.88805333e-03,  1.03343592e+00,  1.60927835e-15,
        3.66966835e-02,  3.66798679e-02,  1.30084935e-01,  3.00075980e-02,
        3.91769452e-02,  1.33064630e-01,  4.55867726e-02,  1.65251144e-02,
       -6.73611249e-03,  4.11383266e-02, -5.61353224e-03,  6.94379425e-02,
        1.18314580e-01,  0.00000000e+00,  3.82737205e-02]), message='Optimization terminated successfully', name_method='SLSQP', name_obj='MLW')

In [152]:
# 13. 모형 적합 결과 및 적합도 지표 확인
stats = calc_stats(m)
print("적합도 지표:\n", stats.T,"\n\n")

estimates = m.inspect()
print("모수 추정치:\n", estimates)

적합도 지표:
                       Value
DoF               13.000000
DoF Baseline      28.000000
chi2             747.249325
chi2 p-value       0.000000
chi2 Baseline  25996.232161
CFI                0.971725
GFI                0.971255
AGFI               0.938089
NFI                0.971255
TLI                0.939100
RMSEA              0.050201
AIC               45.933320
BIC              230.333438
LogLik             0.033340 


모수 추정치:
          lval  op       rval      Estimate  Std. Err     z-value   p-value
0      stress   ~       mood  5.900738e-01  0.008782   67.188245       0.0
1      stress   ~      grade  1.755945e-02  0.008267    2.124072  0.033664
2      stress   ~     family -3.888053e-03  0.008478   -0.458581  0.646535
3     suicide   ~     stress  1.033436e+00  0.017201   60.081688       0.0
4       M_STR   ~     stress  1.000000e+00         -           -         -
5       M_SAD   ~       mood  1.000000e+00         -           -         -
6       M_LON   ~       mood  7.04

In [153]:
# 14. 모형 시각화 및 리포트 생성
semplot(m, './high.png')
report(m, './report')